# RNN学习笔记
## 目录
1. [什么是语言模型？](#1-什么是语言模型)
2. [N-gram语言模型](#2-n-gram语言模型)
3. [固定窗口神经语言模型](#3-固定窗口神经语言模型)
4. [循环神经网络(RNN)语言模型](#4-循环神经网络rnn语言模型)
5. [代码实现](#5-代码实现)
6. [练习题](#8-练习题)

## 1. 什么是语言模型？

###  定义
**语言模型（Language Model）** 是一个概率分布，用于计算一个句子或词序列出现的概率。

### 数学表示
给定一个词序列 $w_1, w_2, ..., w_T$，语言模型计算：

$$P(w_1, w_2, ..., w_T) = \prod_{t=1}^{T} P(w_t | w_1, ..., w_{t-1})$$

## 2. N-gram语言模型

### 基本思想
N-gram模型基于**马尔可夫假设**：当前词只依赖于前面的n-1个词。

### 数学原理

#### Unigram (1-gram)
$$P(w_1, w_2, ..., w_T) = \prod_{t=1}^{T} P(w_t)$$

#### Bigram (2-gram)
$$P(w_1, w_2, ..., w_T) = \prod_{t=1}^{T} P(w_t | w_{t-1})$$

#### Trigram (3-gram)
$$P(w_1, w_2, ..., w_T) = \prod_{t=1}^{T} P(w_t | w_{t-2}, w_{t-1})$$

## 3. 固定窗口神经语言模型

### 3.1 基本思想
使用神经网络来学习词的分布式表示，通过固定大小的上下文窗口预测下一个词。

### 3.2 模型架构
```
输入层：[w_{t-n+1}, ..., w_{t-1}] (n-1个词的one-hot向量)
    
嵌入层：将one-hot向量映射为稠密向量
    
拼接层：将所有词向量拼接
    
隐藏层：全连接层 + 激活函数
    
输出层：Softmax层，输出词汇表上的概率分布
```

### 3.3 数学表示

#### 输入表示
$$x = [e(w_{t-n+1}); e(w_{t-n+2}); ...; e(w_{t-1})]$$

其中 $e(w)$ 是词 $w$ 的嵌入向量，$[;]$ 表示向量拼接。

#### 隐藏层
$$h = \tanh(W_h x + b_h)$$

#### 输出层
$$\hat{y} = \text{softmax}(W_o h + b_o)$$

#### 损失函数
$$L = -\sum_{t} \log P(w_t | w_{t-n+1}, ..., w_{t-1})$$

## 4. 循环神经网络(RNN)语言模型

### 基本思想
RNN通过隐藏状态来维护历史信息，理论上可以捕获任意长度的依赖关系。

###  模型架构
![](img/rnn.png)

###  数学表示

#### 隐藏状态更新
$$h_t = \tanh(W_h h_{t-1} + W_x x_t + b_h)$$

#### 输出计算
$$y_t = \text{softmax}(W_y h_t + b_y)$$

#### 损失函数
$$L = -\sum_{t=1}^{T} \log P(w_t | w_1, ..., w_{t-1})$$

###  RNN的问题

#### 计算效率问题
- **顺序计算**：必须按时间步顺序计算，无法并行化
- **计算慢**：对于长序列，计算时间线性增长

####  梯度消失问题
- **长距离依赖**：随着序列长度增加，早期信息逐渐丢失
- **梯度消失**：反向传播时梯度指数衰减

## N-gram语言模型实现

In [2]:
class NGramLanguageModel:
    def __init__(self, n=2):
        self.n = n
        self.ngram_counts = defaultdict(int)
        self.context_counts = defaultdict(int)
        self.vocab = set()
    
    def train(self, sentences):
        """训练N-gram模型"""
        for sentence in sentences:
            words = ['<START>'] * (self.n - 1) + sentence.lower().split() + ['<END>']
            self.vocab.update(words)
            
            # 统计n-gram和(n-1)-gram的出现次数
            for i in range(len(words) - self.n + 1):
                ngram = tuple(words[i:i + self.n])
                context = tuple(words[i:i + self.n - 1])
                
                self.ngram_counts[ngram] += 1
                self.context_counts[context] += 1
    
    def probability(self, word, context):
        """计算P(word|context)"""
        ngram = context + (word,)
        
        if self.context_counts[context] == 0:
            return 1.0 / len(self.vocab)  # 平滑处理
        
        # 加1平滑
        return (self.ngram_counts[ngram] + 1) / (self.context_counts[context] + len(self.vocab))
    
    def sentence_probability(self, sentence):
        """计算句子概率"""
        words = ['<START>'] * (self.n - 1) + sentence.lower().split() + ['<END>']
        prob = 1.0
        
        for i in range(self.n - 1, len(words)):
            context = tuple(words[i - self.n + 1:i])
            word = words[i]
            prob *= self.probability(word, context)
        
        return prob
    
    def perplexity(self, test_sentences):
        """计算困惑度"""
        total_log_prob = 0
        total_words = 0
        
        for sentence in test_sentences:
            words = sentence.lower().split()
            total_words += len(words) + 1  # +1 for <END>
            prob = self.sentence_probability(sentence)
            total_log_prob += math.log(prob)
        
        return math.exp(-total_log_prob / total_words)

## 固定窗口神经语言模型

In [3]:
class FixedWindowNeuralLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, window_size):
        super(FixedWindowNeuralLM, self).__init__()
        self.window_size = window_size
        self.embedding_dim = embedding_dim
        
        # 词嵌入层
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # 隐藏层
        self.hidden = nn.Linear(window_size * embedding_dim, hidden_dim)
        
        # 输出层
        self.output = nn.Linear(hidden_dim, vocab_size)
        
        # Dropout
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        # x: [batch_size, window_size]
        batch_size = x.size(0)
        
        # 词嵌入
        embeds = self.embedding(x)  # [batch_size, window_size, embedding_dim]
        
        # 拼接所有词向量
        concat_embeds = embeds.view(batch_size, -1)  # [batch_size, window_size * embedding_dim]
        
        # 隐藏层
        hidden = torch.tanh(self.hidden(concat_embeds))
        hidden = self.dropout(hidden)
        
        # 输出层
        output = self.output(hidden)
        
        return output

## RNN语言模型

In [4]:
class RNNLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        super(RNNLanguageModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # 词嵌入层
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # RNN层
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)
        
        # 输出层
        self.output = nn.Linear(hidden_dim, vocab_size)
        
        # Dropout
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x, hidden=None):
        # x: [batch_size, seq_len]
        batch_size = x.size(0)
        
        # 初始化隐藏状态
        if hidden is None:
            hidden = self.init_hidden(batch_size)
        
        # 词嵌入
        embeds = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        
        # RNN前向传播
        rnn_out, hidden = self.rnn(embeds, hidden)  # [batch_size, seq_len, hidden_dim]
        
        # Dropout
        rnn_out = self.dropout(rnn_out)
        
        # 输出层
        output = self.output(rnn_out)  # [batch_size, seq_len, vocab_size]
        
        return output, hidden
    
    def init_hidden(self, batch_size):
        """初始化隐藏状态"""
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim)

## 6. 练习题(由AI总结应该掌握的知识点）

### 理论题

1. **基础概念**
   - 解释语言模型的定义和作用
   - 什么是困惑度？如何计算？
   - 马尔可夫假设在N-gram模型中的作用是什么？

2. **数学推导**
   - 推导Bigram模型的最大似然估计公式
   - 解释为什么需要平滑技术
   - 推导RNN语言模型的梯度计算公式

3. **模型对比**
   - 比较N-gram和神经语言模型的优缺点
   - 为什么RNN会出现梯度消失问题？
   - 固定窗口神经语言模型相比N-gram的改进在哪里？

### 实践题

1. **代码实现**
   - 实现Add-k平滑的N-gram模型
   - 修改RNN模型，使用LSTM替代普通RNN
   - 实现一个简单的文本生成器

2. **实验设计**
   - 在更大的数据集上比较不同模型的性能
   - 分析窗口大小对固定窗口模型性能的影响
   - 实现并比较不同的平滑技术

3. **应用拓展**
   - 将语言模型应用到拼写检查任务
   - 实现一个简单的机器翻译评估器
   - 设计一个基于语言模型的文本分类器